In [6]:
#period_info table
from glob import glob
import pandas as pd
# Find all directories under tennis_data/
date_folders = glob('tennis_data/*')

column_home= ['match_id','statistic_name', 'home_value', 'away_value']
files_to_load_period = []

# Loop through each date folder and load .parquet files
for folder in date_folders:
    files_to_load_period.extend(
        glob(f'{folder}/data/raw/raw_statistics_parquet/*.parquet')
    )
df_period = [pd.read_parquet(file, columns=column_home) for file in files_to_load_period]
df_period = pd.concat(df_period, ignore_index=True)

In [2]:
#matchhometeaminfo tabe
column_home= ['match_id','gender']

# Create an empty list to store all files
files_home= []
# Loop through each date folder and load .parquet files
for folder in date_folders:
    files_home.extend(
    [f for f in glob(f'{folder}/data/raw/raw_match_parquet/home_team*.parquet') if '_score' not in f]
    )
df_home_team_info = [pd.read_parquet(file, columns=column_home) for file in files_home]
df_home_team_info = pd.concat(df_home_team_info, ignore_index=True)

In [3]:
#match_away_team_info table
column_away= ['match_id','gender']
files_away= []
# Loop through each date folder and load .parquet files
for folder in date_folders:
    files_away.extend(
    [f for f in glob(f'{folder}/data/raw/raw_match_parquet/away_team*.parquet') if '_score' not in f]
    )
df_away_team_info = [pd.read_parquet(file, columns=column_away) for file in files_away]
df_away_team_info = pd.concat(df_away_team_info, ignore_index=True)

In [11]:
df_double_faults = df_period[df_period['statistic_name'] == 'double_faults']
df = (
    df_double_faults
    .merge(df_home_team_info[['match_id','gender']], on='match_id', how='left')
    .merge(df_away_team_info[['match_id','gender']], on='match_id', how='left', suffixes=('_home','_away'))
)
home_faults = df.groupby('gender_home')['home_value'].sum()
away_faults = df.groupby('gender_away')['away_value'].sum()
total_faults = home_faults.add(away_faults, fill_value=0)
total_faults

gender_home
F    534385
M    546379
dtype: int64